## __🎧 Audio Diarization Project (pyannote.audio)__

#### 🧱 Requisitos del sistema
- Python 3.12.12
- Cuenta en Hugging Face con acceso a modelos pyannote/

## 📌 Descripción
Este proyecto utiliza el pipeline pyannote/speaker-diarization-3.0 para:
- detectar segmentos de voz
- identificar hablantes distintos
- preparar el audio para cortes posteriores

In [ ]:
# 0. Check python version
!python3 --version

In [ ]:
# 1. Limpiar entorno previo
!pip uninstall -y numpy torch torchaudio pyannote.audio

In [ ]:
# 2. Install ffmpeg
!apt-get update
!apt-get install -y ffmpeg

In [ ]:
# 3. Install main libraries
!pip install "numpy<2.0" \
             "torch==2.3.1" \
             "torchvision==0.18.1" \
             "torchaudio==2.3.1" \
             "onnxruntime" \
             "huggingface-hub>=0.19.4" \
             "pyannote.audio==3.1.1" \
             "pydub" \
             "ipywidgets"

# NOTA IMPORTANTE:
# Al terminar, ve al menú arriba: "Entorno de ejecución" -> "Reiniciar sesión".

In [ ]:
import os
import torch

from pydub import AudioSegment
from pydub.effects import normalize
from pydub.silence import detect_silence

from pyannote.audio import Pipeline
import ipywidgets as widgets

In [ ]:
# Get HF token from Colab userdata
from google.colab import userdata
HF_TOKEN = userdata.get('PYANNOTE_HF_TOKEN')

In [ ]:
# Test diarization pipeline
pipeline = Pipeline.from_pretrained(
   'pyannote/speaker-diarization-3.0',
   use_auth_token=HF_TOKEN
)

In [ ]:
# Main code for diarizarion and segmentation